In [1]:
import asyncio
from pathlib import Path

from IPython.display import display, clear_output
import ipywidgets as widgets
from rdkit import Chem
import nglview as nv

In [2]:
dir_survey = Path.cwd()

In [3]:
dir_pics = Path(dir_survey, "images_2D")

bname_pics = "image_molecule"

In [4]:
dir_confs = Path(dir_survey, "conformers_3D")

bname_confs = "best_rocs_conformer"

In [5]:
n_pairs = 100
ids_pairs = list(range(1, n_pairs + 1))

In [6]:
suffixes_mols = ["a", "b"]

In [7]:
conditions = widgets.Label(value=(f"You will be shown {n_pairs} pairs of molecules."
                                  "\nDo you accept?"),
                           )

accept = widgets.RadioButtons(options=["I Accept", "I Decline"], value="I Decline")

grid_initial = widgets.GridBox([conditions, accept])

In [8]:
info_pair_str = "Pair {}" + f"/{n_pairs}"
info_pair = widgets.Label(info_pair_str)

border_width = "2px"
border_style = "solid"
border_color = "black"
border_attributes = f"{border_width} {border_style} {border_color}"

layout_headers = dict(height="auto",
                      width="auto",
                      )

layout_pic = {"border": border_attributes}
pics = {suffix_mol: widgets.Image(format="svg+xml", layout=layout_pic) for suffix_mol in suffixes_mols}
header_pics = widgets.HTML("2D Representations:", layout=layout_headers)
grid_pics = widgets.AppLayout(left_sidebar=pics["a"],
                              right_sidebar=pics["b"],
                              )

layout_view = {"border": border_attributes}
views = {suffix_mol: nv.NGLWidget(layout=layout_view) for suffix_mol in suffixes_mols}
grid_gap = "1%"
header_views = widgets.Label("3D Representations (interactive):", layout=layout_headers)
layout_reset_button = dict(width="auto")
reset_button = widgets.Button(description="Reset 3D Views", layout=layout_reset_button)
grid_views = widgets.AppLayout(left_sidebar=views["a"],
                               right_sidebar=views["b"],
                               footer=reset_button,
                               grid_gap=grid_gap,
                               )

next_button = widgets.Button(description="Next")


grid_survey = widgets.GridBox([info_pair,
                               next_button,
                               header_pics,
                               grid_pics,
                               header_views,
                               grid_views,
                               ],
                              layout={"display": "none"},
                               )

In [9]:
completed = widgets.Label(f"You visualized all {n_pairs} pairs!",
                          layout={"display": "none"}
                          )

In [10]:
def switch_widget(old, new):
    """Conceal `old` widget and show `new` widget."""
    old.layout.display = "none"
    new.layout.display = "block"

In [11]:
def new_displayed_molecule(mol: Chem.Mol,
                           view: nv.NGLWidget,
                           ) -> nv.component.ComponentViewer:
    """Remove all existing molecules and display a new molecule in a NGLWidget view."""
    
    #Remove all existing components in `view`.
    #(if `view` is empty, no component will be removed.)
    for component in view:
        view.remove_component(component)
    
    #Add a new component, thus displaying `mol`.
    component = view.add_component(mol)
    
    #For a better display in Voilà.
    view.handle_resize()
    
    return component

In [12]:
def wait_for_change(widget, value):
    future = asyncio.Future()
    
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
        return change.new
    
    widget.observe(getvalue, value)
    return future

In [13]:
def wait_for_click(next_button, id_pair):
    future = asyncio.Future()
    
    #(!)Remember to display a widget before changing its value!
    #Otherwise, it will not work as intended (will not appear, will not update...).
    
    nums_atoms = []
    for suffix_mol in suffixes_mols:
        
        #Load 2D picture and display it.
        file_pic = Path(dir_pics, f"{bname_pics}_{id_pair:03}{suffix_mol}.svg")
        pics[suffix_mol].set_value_from_file(file_pic)
        
        #Load 3D conformer.
        file_mol = Path(dir_confs, f"{bname_confs}_{id_pair:03}{suffix_mol}.pdb")
        mol = Chem.MolFromPDBFile(file_mol.as_posix(), removeHs=False)
        
        #Calculate number of atoms
        #(it will be used to define a default orientation).
        nums_atoms.append(mol.GetNumAtoms())
            
        #Display molecule.
        new_displayed_molecule(mol, views[suffix_mol])
        
    
    #Calculate a default orientation for current molecular pair,
    #based on number of atoms.
    max_n_atoms = max(nums_atoms)
    default_orientation = [0 for i in range(15)]
    default_orientation += [1] #last element of 4x4 matrix is always 1
    value_diag = min(30, max_n_atoms)
    ids_diag = [0, 5, 10] #id of diagonal elements (except last)
    for i in ids_diag:
        default_orientation[i] = value_diag
    
    #And set default orientation to both molecules.
    views["a"]._set_camera_orientation(default_orientation)
    views["b"]._set_camera_orientation(default_orientation)
    
    
    #Make moving one molecule also move the other.
    views["a"]._set_sync_camera([views["b"]])
    views["b"]._set_sync_camera([views["a"]])
    
    info_pair.value = info_pair_str.format(id_pair)
    
    
    def reset_views(reset_button):
        views["a"]._set_camera_orientation(default_orientation)
        views["b"]._set_camera_orientation(default_orientation)
    
    reset_button.on_click(reset_views)
    
    
    def on_button_clicked(next_button):
        future.set_result(None)
    
    
    next_button.on_click(on_button_clicked)
    return future

In [14]:
async def survey_molecules(ids_pairs):
    for id_pair in ids_pairs:
        await wait_for_click(next_button, id_pair)

In [15]:
async def f():
    
    await wait_for_change(accept, "value")
    
    switch_widget(grid_initial, grid_survey)
    
    await survey_molecules(ids_pairs)
    
    switch_widget(grid_survey, completed)


task = asyncio.create_task(f())

display(grid_initial,
        grid_survey,
        completed,
        )

GridBox(children=(Label(value='You will be shown 100 pairs of molecules.\nDo you accept?'), RadioButtons(index…

GridBox(children=(Label(value='Pair {}/100'), Button(description='Next', style=ButtonStyle()), HTML(value='2D …

Label(value='You visualized all 100 pairs!', layout=Layout(display='none'))